<a href="https://colab.research.google.com/github/chengyang122/Metter.js-and-MediaPipe-Interactive-game/blob/main/Detectron2EvaluateDecode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!sudo apt-get install libzbar0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libv4l-0 libv4lconvert0
The following NEW packages will be installed:
  libv4l-0 libv4lconvert0 libzbar0
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 193 kB of archives.
After this operation, 760 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libv4lconvert0 amd64 1.14.2-1 [76.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libv4l-0 amd64 1.14.2-1 [41.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libzbar0 amd64 0.10+doc-10.1build2 [75.7 kB]
Fetched 193 kB in 1s (331 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog

In [11]:
pip install pyzbar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install 'git+https://github.com/facebookresearch/detectron2.git'
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
torch:  1.11 ; cuda:  cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-3ozfps08
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-3ozfps08


In [3]:
import numpy as np
import pandas as pd
import cv2
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from tqdm import tqdm
# import some common libraries
import numpy as np
import os, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

from pyzbar import pyzbar
import cv2 as cv

In [4]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("boardetect_train",)
cfg.DATASETS.TEST = ("boardetect_test",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/barcode/checkpoint/model_9991.pth'  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 1000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
cfg.MODEL.DEVICE='cpu'
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9 # set the testing threshold for this model
#cfg.DATASETS.TEST = ("boardetect_test", )
predictor = DefaultPredictor(cfg)
print('done')

[06/28 23:51:20 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                                  | Names in Checkpoint                                                                                  | Shapes                                          |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.bottom_up.res2.0.conv1.*               | backbone.bottom_up.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (256,) (256,) (256,) (256,) (256,64,1,1)        |
| backbone.bottom_up.res2.0.conv2.*               | backbone.bottom_up.res2.0.conv2.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (256,) (256,) (256,) (256,) (256,8,3,3)         |
| backbone.bottom_up.res2.0.conv3.*               | backbone.bottom_up.res2.0.conv3.{norm.bia

In [5]:
def collect_predicted(result):
  Object, Mask = result
  useful_index = []
  for i in range(len(Object)):
    score = Object[i]
    if score > 0.6:
      useful_index.append(i)
  rects = []
  predicted_box = []
  predicted_contour = []
  for index in useful_index:
    mg = Mask[index].cpu().numpy().astype(np.uint8)
    contours, hierarchy = cv2.findContours(mg, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnt = contours[0]
    rect = cv2.minAreaRect(cnt)
    rects.append(rect)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    predicted_box.append(box)
    predicted_contour.append(cnt)
  return predicted_box, predicted_contour, rects

In [6]:
im = cv2.imread('/content/drive/MyDrive/barcode/Ananya.zip (Unzipped Files)/20170614-19_12_58-raspberrypi-camA_undistored.jpg')
outputs = predictor(im)

/usr/local/lib/python3.7/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [7]:
result = (outputs["instances"].scores, outputs["instances"].pred_masks)
predicted_box, predicted_contour,rects = collect_predicted(result)

In [8]:
def crop_rect(img, rect):
    # get the parameter of the small rectangle
    center, size, angle = rect[0], rect[1], rect[2]
    center, size = tuple(map(int, center)), tuple(map(int, size))

    # get row and col num in img
    height, width = img.shape[0], img.shape[1]

    # calculate the rotation matrix
    M = cv2.getRotationMatrix2D(center, angle, 1)
    # rotate the original image
    img_rot = cv2.warpAffine(img, M, (width, height))

    # now rotated rectangle becomes vertical, and we crop it
    img_crop = cv2.getRectSubPix(img_rot, size, center)
    if img_crop.shape[0]>img_crop.shape[1]:
      img_crop = cv2.rotate(img_crop, cv2.cv2.ROTATE_90_CLOCKWISE)
    return img_crop, img_rot

In [33]:
def decode(img_crop):
  (h, w) = img_crop.shape[:2]
  (cX, cY) = (w // 2, h // 2)
  M = cv2.getRotationMatrix2D((cX, cY), 0, 1.0)
  rotated = cv2.warpAffine(img_crop, M, (w, h))

  resized_image = cv2.resize(rotated, (rotated.shape[1]*3, rotated.shape[0]*3)) 
  gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

  img = gray

  closed = cv2.morphologyEx(img, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 21)))

  dens = img[int(img.shape[0]/2)]
  mean = np.mean(dens)

  thresh = closed.copy()
  for idx, val in enumerate(dens):
      if val< mean:
          thresh[:,idx] = 0

  (_, thresh2) = cv2.threshold(thresh, 128, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
  barcodes = pyzbar.decode(thresh2)
  return barcodes

In [20]:
for rect in rects:
  img_crop, img_rot = crop_rect(im, rect)
  decode(img_crop)

[]
[Decoded(data=b'USCSHMLIB56', type='CODE128', rect=Rect(left=94, top=24, width=1475, height=356), polygon=[Point(x=94, y=199), Point(x=95, y=379), Point(x=1569, y=380), Point(x=1569, y=24), Point(x=95, y=25)], quality=81, orientation=None)]
[Decoded(data=b'USCSHMLIB55', type='CODE128', rect=Rect(left=48, top=41, width=1171, height=252), polygon=[Point(x=48, y=41), Point(x=48, y=293), Point(x=1219, y=292), Point(x=1219, y=142), Point(x=1216, y=136)], quality=50, orientation=None)]
[Decoded(data=b'USCSHMLIB48', type='CODE128', rect=Rect(left=73, top=0, width=1174, height=260), polygon=[Point(x=73, y=1), Point(x=73, y=259), Point(x=1247, y=260), Point(x=1247, y=0)], quality=90, orientation=None)]
[Decoded(data=b'USCSHMLIB52', type='CODE128', rect=Rect(left=40, top=135, width=1207, height=185), polygon=[Point(x=40, y=135), Point(x=40, y=139), Point(x=42, y=319), Point(x=1247, y=320), Point(x=1247, y=154), Point(x=1245, y=138), Point(x=1244, y=136)], quality=57, orientation=None)]
[Decod

In [22]:
img_address_base = '/content/drive/MyDrive/barcode/Ananya.zip (Unzipped Files)/'
df1 = pd.read_excel('/content/drive/MyDrive/barcode/AnanyaAnnotation_10-09-2017.xlsx')
df2 = pd.read_excel('/content/drive/MyDrive/barcode/Christian_annotations_2017-10-09.xlsx', header=None)
df2.columns = df1.columns
df2['Image Filename'] = df2['Image Filename'].str[:-4]


frames = [df1, df2]
df = pd.concat(frames, ignore_index=True)

In [35]:
all_detected_barcode = []

for i in df['Image Filename'].unique():
  im = cv2.imread('/content/drive/MyDrive/barcode/Ananya.zip (Unzipped Files)/' + i +'.jpg')
  outputs = predictor(im)
  result = (outputs["instances"].scores, outputs["instances"].pred_masks)
  predicted_box, predicted_contour, rects = collect_predicted(result)

  for rect in rects:
    img_crop, img_rot = crop_rect(im, rect)
  all_detected_barcode.append(decode(img_crop))

AttributeError: ignored